In [1]:
import cuml as cml
import cupy as pd
from numba import cuda
from cuml import ForestInference
import sklearn, numpy as np
from sklearn import datasets
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cuml.test.utils import array_equal

In [2]:
iris = datasets.load_iris()

In [3]:
X = iris.data[:, :2]
y = iris.target

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7,random_state=42)

In [5]:
def train_xgboost_model(X_train, y_train, num_rounds, model_path):
    # set the xgboost model parameters
    params = { 'eval_metric':'error',
              'max_depth': 16}
    dtrain = xgb.DMatrix(X_train, label=y_train)
    # train the xgboost model
    bst = xgb.train(params, dtrain, num_rounds)
    # save the trained xgboost model
    bst.save_model(model_path)
    return bst

In [6]:
def predict_xgboost_model(X_validation, y_validation, xgb_model):
    # predict using the xgboost model
    dvalidation = xgb.DMatrix(X_validation, label=y_validation)
    xgb_preds = xgb_model.predict(dvalidation)
    # convert the predicted values from xgboost into class labels
    xgb_preds = np.around(xgb_preds)
    return xgb_preds

In [7]:
model_path = 'xgb.model'
# num of iterations for which the model is trained
num_rounds = 15

In [8]:
xgboost_model = train_xgboost_model(X_train, y_train, num_rounds, model_path)
predsxgb = predict_xgboost_model(X_val, y_val, xgboost_model)
print(accuracy_score(predsxgb,y_val))

0.7111111111111111


In [9]:
trained_model_preds = predict_xgboost_model(X_val, y_val, xgboost_model)

In [10]:
fm = ForestInference.load(filename=model_path,
                          algo='BATCH_TREE_REORG',
                          output_class=True,
                          threshold=0.50,
                          model_type='xgboost')

In [11]:
preds = fm.predict(X_val)
print(accuracy_score(preds,y_val))

0.6888888888888889


In [12]:
print("The shape of predictions obtained from xgboost : ",(trained_model_preds).shape)
print("The shape of predictions obtained from FIL : ",(preds).shape)
print("Are the predictions for xgboost and FIL the same : " ,   array_equal(trained_model_preds, preds))

The shape of predictions obtained from xgboost :  (45,)
The shape of predictions obtained from FIL :  (45,)
Are the predictions for xgboost and FIL the same :  False
